<a href="https://colab.research.google.com/github/ysb06/dgm-2024-vae-diffusion/blob/main/notebooks/sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DiffuseVAE in Google Colab

Google Colab에서 실행하는 예시

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Git Clone

Github에서 Personal Access Token 발급 후 Clone을 수행합니다. Token은 재사용이 가능하므로 안전한 곳에 저장해 두고 사용하면 됩니다.

### Personal Access Token 발급 방법

1. Github의 Settings - Developer settings (맨아래) 클릭
2. Personal access token 선택
3. Fine-grained tokens 또는 Tokens (classic) 선택
4. 이름, 만료일, 권한 등 설정 후 Generate token
    - Fine-grained tokens의 경우 최소 권한은 Contents만 Read and write로 선택
    - Classic의 경우 최소 권한은 repo만 선택
5. 키 복사 후 사용

In [ ]:
!git clone https://(Github Personal Access Token)@github.com/ysb06/dgm-2024-vae-diffusion.git

## 프로젝트 패키지 설치

Google Colab에서 아래 실행 후, 런타임 세션을 재시작한다. (런타임 해제 및 삭제하면 안 됨)

In [ ]:
%pip install -e dgm-2024-vae-diffusion

## 예시: Baseline 학습 실행

### 하이퍼파라미터 로드

In [ ]:
from hydra import initialize, compose
from omegaconf import DictConfig


def load_config(
    path: str = "dgm-2024-vae-diffusion/src/baseline/configs",
    name: str = "config",
) -> DictConfig:
    with initialize(config_path=path, version_base=None):
        config = compose(config_name=name)
    return config

In [ ]:
config = load_config()
print(config)

### Config 수정

In [ ]:
# VSCode에서 실행하는 것과 다르게 Notebook파일이 있는 폴더가 현재 폴더 위치
config.dataset.ddpm.data.root = "./dgm-2024-vae-diffusion/datasets"
config.dataset.ddpm.training.vae_chkpt_path = "./dgm-2024-vae-diffusion/outputs/vae.pt"
config.dataset.ddpm.training.results_dir = "./dgm-2024-vae-diffusion/outputs"
config.dataset.vae.data.root = "./dgm-2024-vae-diffusion/datasets"
config.dataset.vae.training.results_dir = "./dgm-2024-vae-diffusion/outputs"

config

In [ ]:
import baseline.train_ae as vae_trainer
import baseline.train_ddpm as ddpm_trainer

vae_trainer.train(config)
# ddpm_trainer.train(config)

## Custom Hybrid DiffuseVAE 실행

In [ ]:
import os

config_root = os.path.join("dgm-2024-vae-diffusion", "src/hybrid_vd/configs")

In [ ]:
# config_root = os.path.join("../..", config_root)

In [ ]:
config = load_config(config_root, "train")
print(config)

In [ ]:
config.dataset.root = os.path.join("..", config.dataset.root)
config.trainer.default_root_dir = os.path.join("..", config.trainer.default_root_dir)
config.results_dir = os.path.join("..", config.results_dir)
config.ckpt_path = os.path.join("..", config.ckpt_path) if config.ckpt_path is not None else None

In [ ]:
config.dataset.root = "/content/drive/MyDrive/Colab Notebooks/data"
config.trainer.default_root_dir = "/content/drive/MyDrive/Colab Notebooks/outputs/DiffuseVAE"
config.results_dir = "/content/drive/MyDrive/Colab Notebooks/outputs/DiffuseVAE"
config.ckpt_path = "/content/drive/MyDrive/Colab Notebooks/outputs/DiffuseVAE/checkpoints/diffuse_vae-epoch=100-loss=0.0000.ckpt"

In [ ]:
import hybrid_vd.train as hybrid_trainer

hybrid_trainer.train(config)